In [1]:
import sys
import pandas as pd
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

sys.path.append(project_root)
data_dir = os.path.join(project_root, 'data')

data_file_path = os.path.join(data_dir, 'RADCURE_processed_clinical.csv')
df = pd.read_csv(data_file_path, index_col=0)

df.drop(columns=["Study ID"], inplace=True)
df.rename(columns={'survival_time': 'time', 'death':'event'}, inplace=True)


In [6]:
from jarvais.analyzer import Analyzer
from rich import print

analyzer = Analyzer(
    data=df, 
    output_dir='./survival_outputs/analyzer',
    categorical_columns= [
      "Sex",
      "T Stage",
      "N Stage",
      "Stage",
      "Smoking Status",
      "Disease Site",
      "event",
      "HPV Combined",
      "Chemotherapy"
    ],
    continuous_columns = [
      "time",
      "age at dx",
      "Dose"
    ],
    target_variable='event',
    task='survival'
)

print(analyzer)

analyzer.run()

13:14:57 [warning  ] Date columns not specified. Inferring from remaining columns. [jarvais] call=analyzer.__init__:76


Analyzer(
    AnalyzerSettings(
        output_dir=PosixPath('survival_outputs/analyzer'),
        categorical_columns=[
            'Sex',
            'T Stage',
            'N Stage',
            'Stage',
            'Smoking Status',
            'Disease Site',
            'event',
            'HPV Combined',
            'Chemotherapy'
        ],
        continuous_columns=['time', 'age at dx', 'Dose'],
        date_columns=[],
        task='survival',
        target_variable='event',
        generate_report=True,
        settings_path=None,
        settings_schema_path=None,
        missingness=MissingnessModule(
            categorical_strategy={
                'Sex': 'unknown',
                'T Stage': 'unknown',
                'N Stage': 'unknown',
                'Stage': 'unknown',
                'Smoking Status': 'unknown',
                'Disease Site': 'unknown',
                'event': 'unknown',
                'HPV Combined': 'unknown',
                'Chemotherapy': 'unknown'
            },
            continuous_strategy={'time': 'median', 'age at dx': 'median', 'Dose': 'median'},
            enabled=True
        ),
        outlier=OutlierModule(
            categorical_strategy={
                'Sex': 'frequency',
                'T Stage': 'frequency',
                'N Stage': 'frequency',
                'Stage': 'frequency',
                'Smoking Status': 'frequency',
                'Disease Site': 'frequency',
                'event': 'frequency',
                'HPV Combined': 'frequency',
                'Chemotherapy': 'frequency'
            },
            continuous_strategy={'time': 'none', 'age at dx': 'none', 'Dose': 'none'},
            threshold=0.01,
            enabled=True,
            categorical_mapping={}
        ),
        encoding=OneHotEncodingModule(
            columns=[
                'Sex',
                'T Stage',
                'N Stage',
                'Stage',
                'Smoking Status',
                'Disease Site',
                'HPV Combined',
                'Chemotherapy'
            ],
            target_variable='event',
            prefix_sep='|',
            enabled=True
        ),
        visualization=VisualizationModule(
            plots=['corr', 'pairplot', 'umap', 'frequency_table', 'multiplot', 'kaplan_meier'],
            enabled=True
        )
    )
)

         [info     ] Performing missingness analysis... [jarvais] call=missingness.__call__:43
         [info     ] Performing outlier analysis... [jarvais] call=outlier.__call__:60
13:14:58 [info     ] Plotting Correlation Matrix... [jarvais] call=visualization.__call__:115


+-----------------------+-------------------+-----------+-------------+
|                       |                   | Missing   | Overall     |
+=======================+===================+===========+=============+
| n                     |                   |           | 3346        |
+-----------------------+-------------------+-----------+-------------+
| time, mean (SD)       |                   | 0         | 4.1 (2.7)   |
+-----------------------+-------------------+-----------+-------------+
| age at dx, mean (SD)  |                   | 0         | 62.3 (11.6) |
+-----------------------+-------------------+-----------+-------------+
| Dose, mean (SD)       |                   | 0         | 66.7 (5.8)  |
+-----------------------+-------------------+-----------+-------------+
| Sex, n (%)            | Female            |           | 686 (20.5)  |
+-----------------------+-------------------+-----------+-------------+
|                       | Male              |           | 2660 (

         [info     ] Plotting Pairplot...           [jarvais] call=visualization.__call__:118
13:15:00 [info     ] Plotting UMAP...               [jarvais] call=visualization.__call__:124
13:15:06 [info     ] Plotting Frequency Table...    [jarvais] call=visualization.__call__:121
13:15:16 [info     ] Plotting Kaplan Meier Curves... [jarvais] call=visualization.__call__:127
13:15:19 [info     ] Plotting Multiplot...          [jarvais] call=visualization.__call__:136
Font MPDFAA+Inter28ptBold is missing the following glyphs: '
' (\n)


In [7]:
from jarvais.trainer import TrainerSupervised

analyzer.data['event'] = analyzer.data['event'].astype(int)

trainer = TrainerSupervised(
    output_dir="./outputs/trainer", 
    target_variable=['event','time'], 
    task="survival",
)
    
print(trainer)

trainer.run(analyzer.data)

TrainerSupervised(
    TrainerSettings(
        output_dir=PosixPath('outputs/trainer'),
        target_variable=['event', 'time'],
        task='survival',
        stratify_on=None,
        test_size=0.2,
        random_state=42,
        explain=False,
        reduction_module=FeatureReductionModule(method=None, task='survival', keep_k=2, enabled=True),
        trainer_module=SurvivalTrainerModule(
            output_dir=PosixPath('outputs/trainer'),
            classical_models=['CoxPH', 'RandomForest', 'GradientBoosting', 'SVM'],
            deep_models=['MTLR', 'DeepSurv'],
            eval_metric='c_index',
            random_seed=42
        )
    )
)

13:15:34 [info     ] Skipping feature reduction.    [jarvais] call=feature_reduction.__call__:39
         [info     ] Training MTLR...               [jarvais] call=train.train_mtlr:66
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
[W 2025-06-09 13:15:35,681] Trial 1 failed with parameters: {'C1': 100.0, 'dropout': 0.2873687420594126, 'dims': [512, 512]} because of the following error: The value nan is not acceptable.
[W 2025-06-09 13:15:35,682] Trial 1 failed with value nan.
13:16:24 [info     ] Best trial: C1: 0.01, dropout: 0.38638712110505574, dims: [32, 32] [jarvais] call=train.train_mtlr:70
13:16:25 [info     ] Training DeepSurv...           [jarvais] call=train.train_deepsurv:54
13:16:57 [info     ] Best trial: l2_reg: 0.0062


Model Leaderboard
----------------
+------------------+----------------+----------------+----------------+
| model            | test_score     | val_score      | train_score    |
+==================+================+================+================+
| MTLR             | C_INDEX: 0.723 | C_INDEX: 0.744 | C_INDEX: 0.776 |
+------------------+----------------+----------------+----------------+
| CoxPH            | C_INDEX: 0.721 | N/A            | C_INDEX: 0.777 |
+------------------+----------------+----------------+----------------+
| RandomForest     | C_INDEX: 0.716 | N/A            | C_INDEX: 0.853 |
+------------------+----------------+----------------+----------------+
| SVM              | C_INDEX: 0.716 | N/A            | C_INDEX: 0.783 |
+------------------+----------------+----------------+----------------+
| GradientBoosting | C_INDEX: 0.715 | N/A            | C_INDEX: 0.79  |
+------------------+----------------+----------------+----------------+
| DeepSurv         | C_INDEX

In [8]:
from jarvais.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()

⚠️  **Possible Bias Detected in Smoking Status** ⚠️
=== Subgroup Analysis for 'Smoking Status' Using Cox Proportional Hazards Model ===

Model Statistics:
    AIC (Partial):               2375.84
    Log-Likelihood:              -1183.92
    Log-Likelihood Ratio p-value: 0.0000
    Concordance Index (C-index):   0.61
Model Coefficients:
    +---------------------------+---------------+------------------+
    | Feature                   |   Coefficient |   Standard Error |
    +===========================+===============+==================+
    | Smoking Status_Current    |         0.508 |            4.720 |
    +---------------------------+---------------+------------------+
    | Smoking Status_Ex-smoker  |         0.070 |            4.720 |
    +---------------------------+---------------+------------------+
    | Smoking Status_Non-smoker |        -0.710 |            4.721 |
    +---------------------------+---------------+------------------+
    | Smoking Status_unknown    |       